# Exercise 4: Multi-Agent Handoff — Solution

Complete implementation of the multi-agent handoff workflow with Temporal orchestration.

## Prerequisites

Confirm the Temporal dev server is running and `.env` is configured.

In [ ]:
%pip install --quiet temporalio openai rich

import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.common import RetryPolicy
from temporalio.worker import Worker

console = Console()


def get_weather(location: str) -> str:

    weather_data = {        return result

        "San Francisco": "sunny, 72°F",        workflow.logger.info("✅ Multi-agent workflow completed")

        "New York": "cloudy, 65°F",

        "London": "rainy, 58°F",            result = "Query routed to general agent (not implemented in this exercise)"

        "Tokyo": "clear, 70°F",        else:

    }            )

    weather = weather_data.get(location, "partly cloudy, 68°F")                retry_policy=RetryPolicy(maximum_attempts=3),

    return f"The weather in {location} is {weather}"                start_to_close_timeout=timedelta(seconds=30),

                args=[query, context],

                time_agent,

def get_time(location: str) -> str:            result = await workflow.execute_activity(

    time_data = {        elif agent_to_use == "time_agent":

        "San Francisco": "10:45 AM PST",            )

        "New York": "1:45 PM EST",                retry_policy=RetryPolicy(maximum_attempts=3),

        "London": "6:45 PM GMT",                start_to_close_timeout=timedelta(seconds=30),

        "Tokyo": "3:45 AM JST",                args=[query, context],

    }                weather_agent,

    time = time_data.get(location, "12:00 PM")            result = await workflow.execute_activity(

    return f"The current time in {location} is {time}"        if agent_to_use == "weather_agent":



        workflow.logger.info(f"   Routing to: {agent_to_use}")

@activity.defn

async def triage_query(query: str) -> str:        )

    activity.logger.info(f"🔍 Triaging query: {query}")            ),

                initial_interval=timedelta(seconds=1),

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))                maximum_attempts=3,

    response = client.chat.completions.create(            retry_policy=RetryPolicy(

        model="gpt-4o-mini",            start_to_close_timeout=timedelta(seconds=10),

        messages=[            args=[query],

            {            triage_query,

                "role": "system",        agent_to_use = await workflow.execute_activity(

                "content": (

                    "You are a query classifier. Classify queries into one of: "        context = {"history": [], "trace_id": trace_id}

                    "'weather_agent', 'time_agent', or 'general_agent'. "

                    "Respond with ONLY the agent name, nothing else."        workflow.logger.info(f"   Trace ID: {trace_id}")

                ),        workflow.logger.info(f"   Query: {query}")

            },        workflow.logger.info("🚀 Multi-agent workflow started")

            {"role": "user", "content": query},    async def run(self, query: str, trace_id: str) -> str:

        ],    @workflow.run

    )class MultiAgentWorkflow:

@workflow.defn

    agent_name = response.choices[0].message.content.strip()

    activity.logger.info(f"   → Routed to: {agent_name}")

    return agent_name    return result

    activity.logger.info("✅ Time agent completed")



@activity.defn        result = response_message.content

async def weather_agent(query: str, context: dict) -> str:    else:

    activity.logger.info("🌤️  Weather agent handling query")        result = final_response.choices[0].message.content

        )

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))            model="gpt-4o-mini", messages=messages

    tools = [        final_response = client.chat.completions.create(

        {

            "type": "function",            )

            "function": {                }

                "name": "get_weather",                    "content": function_response,

                "description": "Get current weather for a location",                    "name": "get_time",

                "parameters": {                    "role": "tool",

                    "type": "object",                    "tool_call_id": tool_call.id,

                    "properties": {                {

                        "location": {            messages.append(

                            "type": "string",            function_response = get_time(**function_args)

                            "description": "City name",            function_args = eval(tool_call.function.arguments)

                        }        for tool_call in tool_calls:

                    },        messages.append(response_message)

                    "required": ["location"],    if tool_calls:

                },

            },    tool_calls = response_message.tool_calls

        }    response_message = response.choices[0].message

    ]

    )

    messages = [        model="gpt-4o-mini", messages=messages, tools=tools

        {"role": "system", "content": "You are a weather specialist assistant."},    response = client.chat.completions.create(

        {"role": "user", "content": query},

    ]    ]

        {"role": "user", "content": query},

    response = client.chat.completions.create(        {"role": "system", "content": "You are a time specialist assistant."},

        model="gpt-4o-mini", messages=messages, tools=tools    messages = [

    )

    ]

    response_message = response.choices[0].message        }

    tool_calls = response_message.tool_calls            },

                },

    if tool_calls:                    "required": ["location"],

        messages.append(response_message)                    },

        for tool_call in tool_calls:                        }

            function_args = eval(tool_call.function.arguments)                            "description": "City name",

            function_response = get_weather(**function_args)                            "type": "string",

            messages.append(                        "location": {

                {                    "properties": {

                    "tool_call_id": tool_call.id,                    "type": "object",

                    "role": "tool",                "parameters": {

                    "name": "get_weather",                "description": "Get current time for a location",

                    "content": function_response,                "name": "get_time",

                }            "function": {

            )            "type": "function",

        {

        final_response = client.chat.completions.create(    tools = [

            model="gpt-4o-mini", messages=messages    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        )

        result = final_response.choices[0].message.content    activity.logger.info("🕐 Time agent handling query")

    else:async def time_agent(query: str, context: dict) -> str:

        result = response_message.content@activity.defn



    activity.logger.info("✅ Weather agent completed")
    return result

## Run the Solution

Execute the solution workflow to observe agent handoffs.

In [ ]:
# Run the multi-agent handoff solution
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 4: Multi-Agent Handoff — Solution[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "multi-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[MultiAgentWorkflow],
        activities=[triage_query, weather_agent, time_agent],
    ):
        query = "What's the weather like in London and what time is it there?"
        workflow_id = f"multi-agent-{trace_id}"

        console.print(f"[yellow]Query:[/yellow] {query}\n")

        result = await client.execute_workflow(
            MultiAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]✅ Final Response:[/bold green]\n{result}\n")
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    asyncio.run(run_solution())
else:
    import nest_asyncio
    nest_asyncio.apply()
    task = loop.create_task(run_solution())
    await task

## Source Code

The workflow orchestration and activity implementations are defined above in the first code cell.